In [1]:
import itertools as it
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable, PULP_CBC_CMD, value

# Set the possible values of each bit: {0, 1}
a_range = b_range = x_range = y_range = range(2)


def generate_bitstrings(n: int) -> list:
    """
    Function that generates all possible bitstrings of length n.
    """
    if n == 0:
        return [[]]
    else:
        previous_bitstrings = generate_bitstrings(n - 1)
        current_bitstrings = []
        for bitstring in previous_bitstrings:
            current_bitstrings.append(bitstring + [0])
            current_bitstrings.append(bitstring + [1])
        return current_bitstrings

Here, the bitstring lengths that Alice and Bob receive can be set. $x=x\_range^m$ is for Alice, $y=y\_range^n$ is for Bob.

In [2]:
# These variables may be changed
m = 1
n = 1

# These variables shouldn't be changed
# Generates all possible bitstrings for x and y
x_values = generate_bitstrings(m)
y_values = generate_bitstrings(n)

$q(x,y)$ is the distribution function, which returns the probability of Alice receiving $x$ and Bob receiving $y$. For a uniform distribution, `q_uniform` can be called.

In [3]:
def q_uniform():
    x_possibilities = len(x_values)
    y_possibilities = len(y_values)

    return 1 / (x_possibilities * y_possibilities)


# This function may be changed
def q(x: list, y: list):
    """
    :param x: Bitstring x (Alice). A list of integers.
    :param y: Bitstring y (Bob). A list of integers.
    :return: A user-specified probability
    """
    return q_uniform()

Alice and Bob win the game when $a \oplus b = f(x,y)$.
$f(x,y)$ can be set here.

In [4]:
# This function may be changed
def f(x: list, y: list):
    """
    :param x: Bitstring x (Alice). A list of integers.
    :param y: Bitstring y (Bob). A list of integers.
    """
    return x[0] and y[0]


# This function shouldn't be changed
def V(a: int, b: int, x: list, y: list):
    return a ^ b == f(x, y)

Here we start defining the linear program. We start by creating the variables that need to be optimized. After that we add the constraints.


In [5]:
problem = LpProblem(name="non-signalling", sense=LpMaximize)


# Create a dictionary of all variables
variables = {}

for (a, b, x, y) in it.product(a_range, b_range, x_values, y_values):
    variables[f"{a},{b}|{x},{y}"] = LpVariable(name=f"{a},{b}|{x},{y}", lowBound=0, upBound=1)

# Define the objective function: sum_{abxy} q(x,y) V(ab|xy) p(ab|xy)
problem += lpSum([q(x, y) * V(a, b, x, y) * variables[f"{a},{b}|{x},{y}"]
                for (a, b, x, y) in it.product(a_range, b_range, x_values, y_values)])

# Add the constraint \sum_a p(a,b|x,y) = \sum_a p(ab|x'y)
for (b, x, x1, y) in it.product(b_range, x_values, x_values, y_values):
    if x == x1:
        continue

    sum1 = lpSum([variables[f"{a},{b}|{x},{y}"] for a in a_range])
    sum2 = lpSum([variables[f"{a},{b}|{x1},{y}"] for a in a_range])
    problem += sum1 == sum2

# Add the constraint \sum_b p(ab|xy) = \sum_b p(ab|xy')
for (a, x, y, y1) in it.product(a_range, x_values, y_values, y_values):
    if y == y1:
        continue

    sum1 = lpSum([variables[f"{a},{b}|{x},{y}"] for b in b_range])
    sum2 = lpSum([variables[f"{a},{b}|{x},{y1}"] for b in b_range])
    problem += sum1 == sum2

# Add the constraint \sum_{ab} p(ab|xy) = 1
for (x, y) in it.product(x_values, y_values):
    problem += (lpSum([variables[f"{a},{b}|{x},{y}"] for (a, b) in it.product(a_range, b_range)]) == 1)


problem.solve()

print('Sum =', problem.objective.value())
for k, v in variables.items():
    print(f'p({k}) = {value(v)}')

Sum = 1.0
p(0,0|[0],[0]) = 0.5
p(0,0|[0],[1]) = 0.5
p(0,0|[1],[0]) = 0.5
p(0,0|[1],[1]) = 0.0
p(0,1|[0],[0]) = 0.0
p(0,1|[0],[1]) = 0.0
p(0,1|[1],[0]) = 0.0
p(0,1|[1],[1]) = 0.5
p(1,0|[0],[0]) = 0.0
p(1,0|[0],[1]) = 0.0
p(1,0|[1],[0]) = 0.0
p(1,0|[1],[1]) = 0.5
p(1,1|[0],[0]) = 0.5
p(1,1|[0],[1]) = 0.5
p(1,1|[1],[0]) = 0.5
p(1,1|[1],[1]) = 0.0


We can verify that this is indeed the optimal non-signalling strategy by comparing it to the general non-signalling strategy for an XOR game:

$$p(a,b|x,y)=\begin{cases} \frac{1}{2} \hspace{0.3cm}\text{if}\hspace{0.15cm}a\oplus b = f(x,y) \\ 0\hspace{0.35cm}\text{otherwise.} \end{cases}$$